Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [49]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 22.857580
Training accuracy: 11.9%
Validation accuracy: 14.1%
Loss at step 100: 4.937613
Training accuracy: 71.3%
Validation accuracy: 71.0%
Loss at step 200: 4.052884
Training accuracy: 74.7%
Validation accuracy: 73.8%
Loss at step 300: 3.461765
Training accuracy: 76.1%
Validation accuracy: 74.7%
Loss at step 400: 3.011023
Training accuracy: 77.1%
Validation accuracy: 75.1%
Loss at step 500: 2.651141
Training accuracy: 77.9%
Validation accuracy: 75.7%
Loss at step 600: 2.355963
Training accuracy: 78.9%
Validation accuracy: 76.1%
Loss at step 700: 2.109709
Training accuracy: 79.7%
Validation accuracy: 76.4%
Loss at step 800: 1.901798
Training accuracy: 80.3%
Validation accuracy: 76.7%
Test accuracy: 84.0%


In [7]:
batch_size = 128
hidden_layer_size = 1024
beta = 1e-3

hidden_layer_graph = tf.Graph()
with hidden_layer_graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
  weights_2 = tf.Variable(
  tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer = tf.nn.relu(layer_1)
  layer_2 = tf.matmul(hidden_layer, weights_2) + biases_2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(layer_2, tf_train_labels)) + beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [8]:
num_steps = 3001

with tf.Session(graph=hidden_layer_graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 674.090149
Minibatch accuracy: 11.7%
Validation accuracy: 27.9%
Minibatch loss at step 500: 202.995926
Minibatch accuracy: 78.1%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 115.503807
Minibatch accuracy: 83.6%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 68.952194
Minibatch accuracy: 79.7%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 41.264503
Minibatch accuracy: 89.8%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 25.464495
Minibatch accuracy: 78.1%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.553634
Minibatch accuracy: 83.6%
Validation accuracy: 86.9%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
num_steps = 3001

with tf.Session(graph=hidden_layer_graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % 5) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 638.281616
Minibatch accuracy: 12.5%
Validation accuracy: 24.3%
Minibatch loss at step 500: 190.248108
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 1000: 115.376823
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 69.970848
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 42.434216
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2500: 25.734838
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 3000: 15.608407
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Test accuracy: 84.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [18]:
batch_size = 128
hidden_layer_size = 1024
beta = 1e-3
keep_prob = 0.99

hidden_layer_graph = tf.Graph()
with hidden_layer_graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
  weights_2 = tf.Variable(
  tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer = tf.nn.dropout(tf.nn.relu(layer_1), keep_prob)
  layer_2 = tf.matmul(hidden_layer, weights_2) + biases_2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(layer_2, tf_train_labels)) + beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [19]:
num_steps = 3001

with tf.Session(graph=hidden_layer_graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 697.408508
Minibatch accuracy: 3.1%
Validation accuracy: 30.2%
Minibatch loss at step 500: 203.004822
Minibatch accuracy: 74.2%
Validation accuracy: 77.8%
Minibatch loss at step 1000: 115.797478
Minibatch accuracy: 84.4%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 68.925117
Minibatch accuracy: 83.6%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 41.235783
Minibatch accuracy: 89.8%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 25.397512
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.571724
Minibatch accuracy: 84.4%
Validation accuracy: 87.0%
Test accuracy: 93.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [56]:
batch_size = 128
hidden_layer_1_size = 1024
hidden_layer_2_size = 512
beta = 0.001
keep_prob = 0.9
starter_learning_rate = 0.5

hidden_layer_graph = tf.Graph()
with hidden_layer_graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_1_size], stddev=0.5))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_1_size]))
    
  weights_2 = tf.Variable(
  tf.truncated_normal([hidden_layer_1_size, hidden_layer_2_size], stddev=math.sqrt(2.0/hidden_layer_1_size)))
  biases_2 = tf.Variable(tf.zeros([hidden_layer_2_size]))
    
  weights_3 = tf.Variable(
  tf.truncated_normal([hidden_layer_2_size, num_labels], stddev=math.sqrt(2.0/hidden_layer_2_size)))
  biases_3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden_layer_1 = tf.nn.dropout(tf.nn.relu(layer_1), 0.75)
  layer_2 = tf.matmul(hidden_layer_1, weights_2) + biases_2
  hidden_layer_2 = tf.nn.dropout(tf.nn.relu(layer_2), 0.95)
  layer_3 = tf.matmul(hidden_layer_2, weights_3) + biases_3

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(layer_3, tf_train_labels)) + beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3))
  
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1, 0.9999, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_3)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf_valid_dataset, 
                        weights_1) + biases_1
                ), 
                weights_2) + biases_2
        ),
        weights_3) + biases_3
    )
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.relu(
            tf.matmul(
                tf.nn.relu(
                    tf.matmul(
                        tf_test_dataset, 
                        weights_1) + biases_1
                ), 
                weights_2) + biases_2
        ),
        weights_3) + biases_3
    )

In [57]:
num_steps = 10001

with tf.Session(graph=hidden_layer_graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 87.476227
Minibatch accuracy: 8.6%
Validation accuracy: 17.6%
Minibatch loss at step 500: 153.506119
Minibatch accuracy: 50.0%
Validation accuracy: 51.6%
Minibatch loss at step 1000: 96.875671
Minibatch accuracy: 68.8%
Validation accuracy: 68.7%
Minibatch loss at step 1500: 62.500202
Minibatch accuracy: 73.4%
Validation accuracy: 76.8%
Minibatch loss at step 2000: 41.079815
Minibatch accuracy: 85.9%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 27.964003
Minibatch accuracy: 77.3%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 19.324146
Minibatch accuracy: 75.8%
Validation accuracy: 84.3%
Minibatch loss at step 3500: 13.538800
Minibatch accuracy: 82.8%
Validation accuracy: 85.6%
Minibatch loss at step 4000: 9.900564
Minibatch accuracy: 78.9%
Validation accuracy: 85.7%
Minibatch loss at step 4500: 7.101256
Minibatch accuracy: 88.3%
Validation accuracy: 87.2%
Minibatch loss at step 5000: 5.347253
Minibatch accuracy: 83.6%
Validation